In [11]:
%pip install -U langchain-anthropic langchain-core

In [ ]:
import json
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import JsonOutputParser # Changed from JsonOutputKeyParser

# ---------------------------
# SETUP: Claude Model (Sonnet or Opus)
# ---------------------------
claude = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",  # Or use "claude-3-opus-20240229"
    temperature=0.3,
    max_tokens=1024,
    api_key="sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your key
)

# ---------------------------
# PROMPT TEMPLATE
# ---------------------------
prompt = PromptTemplate(
    input_variables=["employee_json"],
    template="""
You are an HR AI agent that helps assess employee churn risk and recommends actions.

Given this JSON input for an employee:
{employee_json}

Analyze and return your structured response in the following JSON format:

{{
  "employee_id": <int>,
  "status": "HIGH RISK" | "LOW RISK",
  "reasons": [ ... ],
  "recommended_actions": [ ... ],
  "summary": "<short paragraph summary>"
}}

Be concise, accurate, and strictly return valid JSON.
"""
)

# ---------------------------
# LLM Chain
# ---------------------------
from langchain_core.runnables import RunnableLambda

chain = prompt | claude | JsonOutputParser() # Changed from JsonOutputKeyParser(key="content")

# ---------------------------
# Load Predictions
# ---------------------------
with open("/content/drive/MyDrive/Cog_Culture_Assignment_UserCase2(NAMAN SONI 21CSU438)/Project_Risk_Agent/churn_predictions.json", "r") as f:
    predictions = json.load(f)

# Add employee IDs
for idx, p in enumerate(predictions):
    p["employee_id"] = idx

# ---------------------------
# Run through LLM and collect outputs
# ---------------------------
results = []
for entry in predictions:
    employee_json = json.dumps(entry, indent=2)
    try:
        output = chain.invoke({"employee_json": employee_json})
        results.append(output)
    except Exception as e:
        print(f"Error with employee {entry['employee_id']}: {e}")
        continue

# ---------------------------
# Save to output file
# ---------------------------
with open("churn_llm_decisions.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"✅ Completed. {len(results)} results saved to churn_llm_decisions.json")

✅ Completed. 100 results saved to churn_llm_decisions.json
